In [23]:
import pandas as pd
import numpy as np
from datetime import timedelta
from pandas_ta.trend import adx
from pandas_ta.trend import psar
import matplotlib.pyplot as plt


import os
import pickle
from config import BASE_DIR
print(os.getcwd())
os.chdir(BASE_DIR)
import scripts.functions as fn 




C:\Users\lbisa\Mi unidad (leonardo.bisaro@gmail.com)\Cripto\python\trd_app


In [24]:
klines_file  = './backtest/klines/2d01/Full_BTCUSDT_2d01_2019-01-01_2024-10-15.DataFrame'
#klines_file  = './backtest/klines/1h01/Full_BTCUSDT_1h01_2019-01-01_2024-10-15.DataFrame'

with open(klines_file, 'rb') as file:
    df = pickle.load(file)
    #df.set_index('datetime',inplace=True)

#df = fn.ohlc_mirror_v(df)

#Filtrar rango de fechas
#ini = pd.to_datetime('2018-06-26')
#fin = pd.to_datetime('2028-05-01')
#df = df[(df['datetime']>=ini)&(df['datetime']<=fin)]
#df.reset_index(inplace=True)
#df.drop(columns=['index'],inplace=True)


#df.drop(columns=['volume'],inplace=True)

#Quitar parte del dataframe
#df = df[500:1500]
#df.reset_index(inplace=True)
#df.drop(columns=['index'],inplace=True)


In [25]:
#Configuracion propuesta por ChatGPT
adx_period = 14
sma_period = 14
atr_period = 14
bollinger_period = 20
atr_threshold = 0.5  # Valor de umbral para ATR
bollinger_threshold = 0.02  # Distancia mínima entre bandas para detectar lateralidad
volume_threshold_factor = 0.5  # Factor para definir el umbral de volumen bajo

In [26]:
# Calcular ATR
def calculate_atr(df, period=atr_period):
    df['SMA'] = df['close'].rolling(window=period).mean()
    df['high-low'] = df['high'] - df['low']
    df['high-close'] = abs(df['high'] - df['close'].shift(1))
    df['low-close'] = abs(df['low'] - df['close'].shift(1))
    df['true_range'] = df[['high-low', 'high-close', 'low-close']].max(axis=1)
    df['ATR'] = (df['true_range']/df['SMA'])*100
    df.drop(['high-low', 'high-close', 'low-close', 'true_range'], axis=1, inplace=True)
    return df

# Calcular Bandas de Bollinger
def calculate_bollinger_bands(df, period=bollinger_period):
    df['SMA'] = df['close'].rolling(window=period).mean()
    df['std_dev'] = df['close'].rolling(window=period).std()
    df['upper_band'] = df['SMA'] + (df['std_dev'] * 2)
    df['lower_band'] = df['SMA'] - (df['std_dev'] * 2)
    df['band_width'] = ((df['upper_band'] - df['lower_band'])/df['SMA'])*100
    df.drop(['std_dev','upper_band','lower_band'], axis=1, inplace=True)
    return df

# Calcular umbral de volumen bajo
def calculate_volume_threshold(df, period=20):
    df['vol_avg'] = df['volume'].rolling(window=period).mean()
    df['volume_threshold'] = volume_threshold_factor * df['vol_avg']
    return df

def calculate_adx(df, period=14):
    tmp = adx(df['high'], df['low'], df['close'], length=period)
    df['ADX'] = tmp[f'ADX_{period}']
    df['DI+'] = tmp[f'DMP_{period}']
    df['DI-'] = tmp[f'DMN_{period}']
    
    return df

def calculate_psar(df):
    tmp = psar(df['high'], df['low'], df['close'], af0=0.02, af=0.4, max_af=None)
    df['PSARl'] = tmp.filter(like='PSARl').iloc[:, 0]
    df['PSARs'] = tmp.filter(like='PSARs').iloc[:, 0]
    return df

In [41]:
# Parámetros configurables
adx_period = 14
sma_period = 14
atr_period = 14
bollinger_period = 20
atr_threshold = 0.5  # Valor de umbral para ATR
bollinger_threshold = 0.05  # Distancia mínima entre bandas para detectar lateralidad
volume_threshold_factor = 0.7  # Factor para definir el umbral de volumen bajo

# Aplicar cálculos
df = calculate_atr(df,atr_period)
df = calculate_bollinger_bands(df,bollinger_period)
df = calculate_adx(df,adx_period)
df = calculate_volume_threshold(df)
df = calculate_psar(df)

df['ADX_cres'] = np.where(df['ADX']>df['ADX'].ewm(span=21).mean(),1,0)
df['trend'] = np.where((df['ADX']> 40) & (df['DI+']>df['DI-']),1,None)   #& (df['ADX_cres']==1) 
df['trend'] = np.where((df['ADX']> 40) & (df['DI+']<df['DI-']),-1,df['trend'])   #& (df['ADX_cres']==1) 
df['buy_limit']  = df['PSARs'] #np.where(df['trend']==-1,   ,None)
df['sell_limit'] = df['PSARl'] #np.where(df['trend']==1,   ,None)
for row in df.iterrows():
    a = row


df['buy']  = None
df['sell'] = None

In [42]:



indicators = [
     {'col': 'buy_limit','name': 'buy_limit','color': 'green','row': 1, 'mode':'markers','symbol':'x',},
     {'col': 'sell_limit','name': 'sell_limit','color': 'red','row': 1,  'mode':'markers','symbol':'x',},
     #{'col': 'ATR','name': 'ATR','color': 'red','row': 1, 'mode':'lines',},
     #{'col': 'st_high','name': 'ST High','color': 'green','row': 1, 'mode':'markers',},
     #   {'col': 'FL_0.0%', 'name': 'FL_0.0%', 'color': 'red', 'row': 1, 'mode': 'lines', }, 
     #   {'col': 'FL_gold1', 'name': 'FL_gold1', 'color': 'orange', 'row': 1, 'mode': 'lines', }, 
     #   {'col': 'FL_gold2', 'name': 'FL_gold2', 'color': 'orange', 'row': 1, 'mode': 'lines', }, 
     #   {'col': 'FL_100.0%', 'name': 'FL_100.0%', 'color': 'green', 'row': 1, 'mode': 'lines', }, 
        ]

indicators_out = [
     {'col': 'ATR','name': 'ATR','color': 'yellow','row': 3, 'mode':'lines',},
     {'col': 'band_width','name': 'band_width','color': 'green','row': 2, 'mode':'lines',},
     {'col': 'ADX','name': 'ADX','color': 'yellow','row': 1, 'mode':'lines',},
     {'col': 'DI+','name': 'DI+','color': 'green','row': 1, 'mode':'lines',},
     {'col': 'DI-','name': 'DI-','color': 'red','row': 1, 'mode':'lines',},
        ]

#https://plotly.com/python/reference/scatter/#scatter-marker-symbol
events = [
       {'df':df,'col':'buy','name': 'BUY','color': 'green','symbol': 'triangle-up' }, 
       {'df':df,'col':'sell','name': 'SELL','color': 'red','symbol': 'triangle-down' }, 
     ]


fig = fn.ohlc_chart(df,indicators=indicators,indicators_out=indicators_out,events=events,) 
fig.show()